In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/Colab/資料分析期末') #切換該目錄
os.listdir() #確認目錄內容
# always needed
import math, os, random, csv
import pandas as pd
import numpy as np

# log and save
import json, logging, pickle, sys, shutil, copy
# torch
import torch
import torch.nn
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, ReLU
import torchvision
from tqdm.auto import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader,ConcatDataset
from torchvision import datasets, models, transforms
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter
%matplotlib inline
import seaborn as sns

# others
import matplotlib.pyplot as plt
from PIL import Image

# sklearn
from sklearn import preprocessing

# statistics
from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statistics

# seeds
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Mounted at /content/drive
Using device: cuda


In [2]:
from sklearn.multioutput import MultiOutputRegressor
import xgboost  as xgb
import  pandas  as  pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [3]:
X = np.load('share_data/finaldura33-22/duration_data_scaled.npy')
y = np.load('share_data/finaldura33-22/train_duration_target.npy')
#train_X, test_X, train_y, test_y = train_test_split(X,y,test_size=0.2, random_state=44)

In [4]:
real_test = np.load('share_data/finaldura33-22/test_duration_data_scaled.npy')
real_test.shape

(7616, 1036)

In [46]:
# feature importance added
#https://discuss.xgboost.ai/t/feature-weights-does-not-work-as-expected/1998
arr_1 = np.repeat(1, 924)
arr_2 = np.repeat(6, 28)
arr_3 = np.repeat(10, 28)
arr_6 = np.repeat(20, 28)
arr_10 = np.repeat(36, 28)

feature_weights = np.concatenate([arr_1, arr_2, arr_3, arr_6, arr_10]).astype(np.float32)

In [121]:
#https://xgboost.readthedocs.io/en/stable/parameter.html
params = {'learning_rate': 0.008, 'n_estimators': 650, 'max_depth': 8, 'min_child_weight': 3.5,
                'seed': 64, 'subsample': 0.65, 'colsample_bytree': 0.7, 'gamma': 0.001, 'reg_alpha': 0, 'reg_lambda': 11,
                'tree_method':'gpu_hist'}
# multioutputregressor = MultiOutputRegressor(xgb.XGBRegressor(objective='binary:logistic', 
#                                                              eval_metric='auc',**params)).fit(train_X, train_y)

In [122]:
xgboost_model = xgb.XGBRegressor(objective='binary:logistic', eval_metric='auc', **params)
multioutputregressor = MultiOutputRegressor(xgboost_model)
multioutputregressor.fit(X, y, verbose=False, feature_weights=feature_weights)
#best_iteration = multioutputregressor.estimators_[0].best_iteration

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.7,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='auc',
                                            feature_types=None, gamma=0.001,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.008, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=8,
                                            max_leaves=None,
                                            min_child_weight=3.5, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=650, n_jobs=None,
                                            num_parallel_tree=None,
                                            objective='binary:logistic',
                                            predictor=None, ...))

In [125]:
y = torch.from_numpy(y).to(torch.float32)

In [126]:
pred=multioutputregressor.predict(X)
pred = torch.from_numpy(pred).to(torch.float32)

In [127]:
#0.285
criterion = nn.BCELoss()
bce_loss = criterion(pred, y)
print("BCE Loss:", bce_loss.item())

BCE Loss: 0.23527812957763672


In [117]:
#test_y = torch.from_numpy(test_y).to(torch.float32)
#test_y = test_y.to(torch.float32)

In [128]:
pred.shape

torch.Size([30460, 28])

In [132]:
#pred = pred.numpy()
np.save('./share_data/XGB/xgb_train.npy', pred) #預測結果

In [133]:
real_test_pred = multioutputregressor.predict(real_test)
real_test_pred

array([[0.2406343 , 0.15595692, 0.03080287, ..., 0.13822316, 0.0217509 ,
        0.35968685],
       [0.17597315, 0.6342018 , 0.02819975, ..., 0.33507273, 0.02345025,
        0.01462035],
       [0.29435802, 0.2899624 , 0.05596036, ..., 0.22746655, 0.02424269,
        0.02512402],
       ...,
       [0.5558023 , 0.56680894, 0.05096456, ..., 0.1631452 , 0.02587575,
        0.01471526],
       [0.08125082, 0.12260158, 0.03509384, ..., 0.10748569, 0.02534017,
        0.01434204],
       [0.11297204, 0.25047067, 0.06266914, ..., 0.12404451, 0.02852888,
        0.01460086]], dtype=float32)

In [134]:
real_test_pred.shape

(7616, 28)

In [136]:
np.save('./share_data/XGB/xgb_test.npy', real_test_pred) #預測結果

In [137]:
showans = pd.DataFrame(real_test_pred)
showans

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0.240634,0.155957,0.030803,0.371932,0.196645,0.127721,0.022314,0.641857,0.047845,0.108438,...,0.019664,0.799170,0.054688,0.107917,0.018088,0.650522,0.061369,0.138223,0.021751,0.359687
1,0.175973,0.634202,0.028200,0.019323,0.117679,0.384926,0.016556,0.015129,0.069233,0.361390,...,0.017861,0.014846,0.065514,0.319642,0.017176,0.015651,0.060438,0.335073,0.023450,0.014620
2,0.294358,0.289962,0.055960,0.053191,0.181930,0.410148,0.028897,0.038161,0.283293,0.299818,...,0.028956,0.061630,0.196106,0.197865,0.018689,0.049076,0.163822,0.227467,0.024243,0.025124
3,0.076075,0.076721,0.027175,0.019679,0.080448,0.094072,0.019009,0.013941,0.138571,0.103383,...,0.018829,0.014364,0.133883,0.101704,0.016766,0.015003,0.112372,0.089404,0.027297,0.013591
4,0.065271,0.101747,0.040246,0.017586,0.103083,0.117146,0.036133,0.015980,0.170789,0.183391,...,0.026196,0.015308,0.084217,0.115556,0.026826,0.014475,0.076431,0.118257,0.030771,0.013703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7611,0.093258,0.124655,0.034760,0.045708,0.108856,0.138455,0.021990,0.015697,0.052771,0.113716,...,0.018827,0.013798,0.050944,0.099155,0.017871,0.014407,0.048537,0.106530,0.025329,0.019642
7612,0.555078,0.555682,0.050965,0.044286,0.353280,0.416680,0.022463,0.023174,0.205132,0.343387,...,0.019799,0.014861,0.167045,0.183122,0.018146,0.016737,0.143964,0.167760,0.025532,0.014625
7613,0.555802,0.566809,0.050965,0.044585,0.338530,0.417227,0.022463,0.022964,0.159405,0.314778,...,0.019377,0.016490,0.096838,0.179927,0.018146,0.016515,0.119213,0.163145,0.025876,0.014715
7614,0.081251,0.122602,0.035094,0.021045,0.097933,0.139493,0.022058,0.014137,0.052662,0.113416,...,0.018761,0.013639,0.051070,0.103323,0.017958,0.014544,0.051559,0.107486,0.025340,0.014342


In [138]:
sample = pd.read_csv("./data/sample.csv")
answer = sample
for i in range(1,29):
    answer.iloc[:,i] = showans.iloc[:, i-1]
answer.to_csv("submit.csv", index=False)
display(answer)

<ipython-input-138-eebc33ef45df>:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  answer.iloc[:,i] = showans.iloc[:, i-1]


,user_id,time_slot_0,time_slot_1,time_slot_2,time_slot_3,time_slot_4,time_slot_5,time_slot_6,time_slot_7,time_slot_8,...,time_slot_18,time_slot_19,time_slot_20,time_slot_21,time_slot_22,time_slot_23,time_slot_24,time_slot_25,time_slot_26,time_slot_27
0,30460,0.240634,0.155957,0.030803,0.371932,0.196645,0.127721,0.022314,0.641857,0.047845,...,0.019664,0.799170,0.054688,0.107917,0.018088,0.650522,0.061369,0.138223,0.021751,0.359687
1,30461,0.175973,0.634202,0.028200,0.019323,0.117679,0.384926,0.016556,0.015129,0.069233,...,0.017861,0.014846,0.065514,0.319642,0.017176,0.015651,0.060438,0.335073,0.023450,0.014620
2,30462,0.294358,0.289962,0.055960,0.053191,0.181930,0.410148,0.028897,0.038161,0.283293,...,0.028956,0.061630,0.196106,0.197865,0.018689,0.049076,0.163822,0.227467,0.024243,0.025124
3,30463,0.076075,0.076721,0.027175,0.019679,0.080448,0.094072,0.019009,0.013941,0.138571,...,0.018829,0.014364,0.133883,0.101704,0.016766,0.015003,0.112372,0.089404,0.027297,0.013591
4,30464,0.065271,0.101747,0.040246,0.017586,0.103083,0.117146,0.036133,0.015980,0.170789,...,0.026196,0.015308,0.084217,0.115556,0.026826,0.014475,0.076431,0.118257,0.030771,0.013703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7611,38071,0.093258,0.124655,0.034760,0.045708,0.108856,0.138455,0.021990,0.015697,0.052771,...,0.018827,0.013798,0.050944,0.099155,0.017871,0.014407,0.048537,0.106530,0.025329,0.019642
7612,38072,0.555078,0.555682,0.050965,0.044286,0.353280,0.416680,0.022463,0.023174,0.205132,...,0.019799,0.014861,0.167045,0.183122,0.018146,0.016737,0.143964,0.167760,0.025532,0.014625
7613,38073,0.555802,0.566809,0.050965,0.044585,0.338530,0.417227,0.022463,0.022964,0.159405,...,0.019377,0.016490,0.096838,0.179927,0.018146,0.016515,0.119213,0.163145,0.025876,0.014715
7614,38074,0.081251,0.122602,0.035094,0.021045,0.097933,0.139493,0.022058,0.014137,0.052662,...,0.018761,0.013639,0.051070,0.103323,0.017958,0.014544,0.051559,0.107486,0.025340,0.014342
